# Tick data

For optimum results this notebook should be run during the Forex trading session.

In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

Create some Forex contracts:

In [2]:
contracts = [Forex(pair) for pair in 'EURUSD USDJPY GBPUSD USDCHF USDCAD AUDUSD'.split()]

eurusd = contracts[0]

Request streaming ticks for them:

In [3]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

Wait a few seconds for the tickers to get filled.

In [4]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', exchange='IDEALPRO'), time=datetime.datetime(2017, 9, 28, 9, 17, 32, 474621, tzinfo=datetime.timezone.utc), bid=1.17645, bidSize=17670000, ask=1.17655, askSize=21170000, lastSize=0, prevBid=1.1765, prevBidSize=18670000, prevAsk=1.1765, prevAskSize=22170000, high=1.17725, low=1.1721, close=1.1745)

The price of Forex ticks is always nan. To get a midpoint price use ``marketPrice()``.

The tickers are kept
live updated, try this a few times to see if the price changes:

In [5]:
ticker.marketPrice()

1.1764999999999999

The following cell will start a 15 second loop that prints a live updated ticker table.
It is updated with every new network packet that arrives.

In [6]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(columns='symbol bidSize bid ask askSize high low close'.split())
df['symbol'] = [c.symbol + c.currency for c in contracts]
contract2Row = {c: i for (i, c) in enumerate(contracts)}

def onPendingTickers(tickers):
    for t in tickers:
        row = contract2Row[t.contract]
        df.iloc[row, 1:] = (t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
        display(df)        

ib.setCallback('pendingTickers', onPendingTickers)
ib.sleep(60)

,symbol,bidSize,bid,ask,askSize,high,low,close
0,EURUSD,2000000,1.1766,1.17665,16670000,1.17725,1.1721,1.1745
1,USDJPY,6000000,112.82,112.825,8000000,113.205,112.69,112.82
2,GBPUSD,11149000,1.3368,1.3369,6649000,1.34095,1.3343,1.3387
3,USDCHF,1000000,0.97385,0.97395,9700000,0.97595,0.97165,0.97215
4,USDCAD,11200000,1.2476,1.2477,2700000,1.25195,1.2469,1.2476
5,AUDUSD,7000000,0.7823,0.78235,1255000,0.7856,0.77995,0.78485


After an update the new tick data is available in the 'ticks' attribute of the ticker.
The tick data will be cleared before the next update.

To stop the live tick subscriptions:

In [7]:
for contract in contracts:
    ib.cancelMktData(contract)

In [8]:
ib.disconnect()